## start session

In [15]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('Elasticsearch to Pyspark').getOrCreate()

In [3]:
from elasticsearch import Elasticsearch

es = Elasticsearch(hosts=["http://localhost:9200"])

response = es.search(index="cars_new", body={
    "query": {
        "match_all": {}
    },
    "size": 1000  
})

data = [hit["_source"] for hit in response["hits"]["hits"]]

In [4]:
from pyspark.sql import SparkSession

# Spark Session erstellen
spark = SparkSession.builder.appName("Elasticsearch to PySpark").getOrCreate()

df = spark.createDataFrame(data)


#from pyspark.ml.feature import VectorAssembler
#assembler = VectorAssembler(inputCols=["feature1", "feature2"], outputCol="features")
#dataset = assembler.transform(df).select("features", "label")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/28 17:38:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
df.head()

Row(color='silver', fuel_type='petrol', make='renault', mileage(kilometers)='250000.0', priceUSD='550', segment='C', transmission='mechanics', volume(cm3)='1700.0', year='1995')

## read data

In [20]:
training=spark.read.csv('/home/ubuntu/CryptoBot/regression/SalaryData.csv',  header=True, inferSchema=True, nullValue="NULL")

In [21]:
training.show()

+------+---+----------+------+
|  Name|Age|Experience|Salary|
+------+---+----------+------+
| Sunny| 52|        10| 30000|
|   May| 19|         5| 20000|
|   Bob| 35|         4| 40000|
|  Paul| 42|         2| 15000|
|Mahesh| 36|         4| 50000|
| Masha| 35|         5| 20000|
+------+---+----------+------+



In [22]:
training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [23]:
training.columns

['Name', 'Age', 'Experience', 'Salary']

In [25]:
from pyspark.ml.feature import VectorAssembler
featureassembler=VectorAssembler(inputCols=["Age","Experience"],outputCol="Independent Features")


In [26]:
output=featureassembler.transform(training)

In [27]:
output.show()

+------+---+----------+------+--------------------+
|  Name|Age|Experience|Salary|Independent Features|
+------+---+----------+------+--------------------+
| Sunny| 52|        10| 30000|         [52.0,10.0]|
|   May| 19|         5| 20000|          [19.0,5.0]|
|   Bob| 35|         4| 40000|          [35.0,4.0]|
|  Paul| 42|         2| 15000|          [42.0,2.0]|
|Mahesh| 36|         4| 50000|          [36.0,4.0]|
| Masha| 35|         5| 20000|          [35.0,5.0]|
+------+---+----------+------+--------------------+



In [28]:
output.columns

['Name', 'Age', 'Experience', 'Salary', 'Independent Features']

In [29]:
finalized_data=output.select("Independent Features","Salary")

In [30]:
from pyspark.ml.regression import LinearRegression
##traintest split
train_data,test_data=finalized_data.randomSplit([0.75,0.25])
regressor=LinearRegression(featuresCol="Independent Features",labelCol="Salary")
regressor=regressor.fit(train_data)


25/01/28 13:57:12 WARN Instrumentation: [d76307d3] regParam is zero, which might cause numerical instability and overfitting.
25/01/28 13:57:13 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
25/01/28 13:57:13 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [31]:
print(f"Coefficients: {regressor.coefficients}")
print(f"Intercept: {regressor.intercept}")


DenseVector([1185.1852, -5768.7991])

In [ ]:
regressor.save("regression_model")